## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")

In [6]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
updated_applications = application_df.drop(columns=['NAME' , 'EIN'], axis=1)
updated_applications.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [7]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = updated_applications['APPLICATION_TYPE'].value_counts()
application_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Look at APPLICATION_TYPE value counts for binning
reduced_applications = updated_applications['APPLICATION_TYPE'].value_counts()[updated_applications['APPLICATION_TYPE'].value_counts()>10]
reduced_applications

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
Name: APPLICATION_TYPE, dtype: int64

In [9]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = updated_applications['APPLICATION_TYPE'].value_counts(ascending=False).index[8:].tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [10]:
# Look at CLASSIFICATION value counts for binning
classification_count = application_df['CLASSIFICATION'].value_counts()
classification_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [11]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_count[classification_count>1]
classification_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [12]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = application_df['CLASSIFICATION'].value_counts(ascending=False).index[4:].tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     4144
C3000     1918
Name: CLASSIFICATION, dtype: int64

In [13]:
# Convert categorical data to numeric with `pd.get_dummies`
numeric_data = pd.get_dummies(application_df, dtype=float)

In [14]:
# Split our preprocessed data into our features and target arrays
x = numeric_data.drop(columns = ['IS_SUCCESSFUL']).values
y = numeric_data['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, Y_train, Y_test = train_test_split(x,y, random_state=42)

In [15]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = len(X_train[0])
print("Input Features =", number_input_features)
hidden_nodes_layer1 = 100
print("Hidden Nodes - Layer 1 =", hidden_nodes_layer1)
hidden_nodes_layer2 = 50
print("Hidden Nodes - Layer 2 =", hidden_nodes_layer2)
hidden_nodes_layer3 = 20
print("Hidden Nodes - Layer 3 =", hidden_nodes_layer3)
hidden_nodes_layer4 = 20
print("Hidden Nodes - Layer 4 =", hidden_nodes_layer4)

#Keras Sequential Model
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Input Features = 19611
Hidden Nodes - Layer 1 = 100
Hidden Nodes - Layer 2 = 50
Hidden Nodes - Layer 3 = 20
Hidden Nodes - Layer 4 = 20
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               1961200   
                                                                 
 dense_1 (Dense)             (None, 50)                5050      
                                                                 
 dense_2 (Dense)             (None, 20)                1020      
                                                                 
 dense_3 (Dense)             (None, 20)                420       
                                                                 
 dense_4 (Dense)             (None, 1)                 21        
                                                                 
Total params: 1,967,711
Trainable params: 1,967,711


In [17]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
# Train the model
fit_model = nn.fit(X_train_scaled,Y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 9s 11ms/step - loss: 0.0730 - accuracy: 0.9704
Epoch 2/100
804/804 [==============================] - 9s 11ms/step - loss: 0.0708 - accuracy: 0.9704
Epoch 3/100
804/804 [==============================] - 9s 11ms/step - loss: 0.0705 - accuracy: 0.9706
Epoch 4/100
804/804 [==============================] - 9s 11ms/step - loss: 0.0702 - accuracy: 0.9712
Epoch 5/100
804/804 [==============================] - 9s 11ms/step - loss: 0.0703 - accuracy: 0.9710
Epoch 6/100
804/804 [==============================] - 9s 11ms/step - loss: 0.0700 - accuracy: 0.9719
Epoch 7/100
804/804 [==============================] - 9s 11ms/step - loss: 0.0697 - accuracy: 0.9716
Epoch 8/100
804/804 [==============================] - 9s 11ms/step - loss: 0.0700 - accuracy: 0.9710
Epoch 9/100
804/804 [==============================] - 9s 11ms/step - loss: 0.0701 - accuracy: 0.9713
Epoch 10/100
804/804 [==============================] - 9s 11ms/step - loss: 0.069

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,Y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.8809 - accuracy: 0.6767 - 998ms/epoch - 4ms/step
Loss: 0.8808674812316895, Accuracy: 0.6767346858978271


In [22]:
# Export our model to HDF5 file
nn.save('../AlphabetSoupCharity_Optimization3.h5')